In [2]:
import sqlite3
import pandas as pd
import numpy as np
from urllib.parse import urlparse
from langdetect import detect
# from zipfile import ZipFile
# from collections import Counter
import re

In [30]:
# Input
db_path = '../Data/moviewreviews.db'
db_name = 'moviereviews'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

df = pd.read_sql_query("SELECT * from " + str(db_name), conn)
print("ACTION: raw data:")
print(df.describe())

print("ACTION: without duplicate rows: ")
df.dropna()
print(df.describe())

print("ACTION: drop rows where any of the values is null or invalid")
df = df[df.REVIEWBODY.notnull()&df.RATING.notnull()&df.REVIEWRATING.notnull()&df.BESTRATING.notnull()&df.WORSTRATING.notnull()]
df = df[~df['REVIEWRATING'].str.contains("ull")]
df = df[~df['BESTRATING'].str.contains("ull")]
df = df[~df['WORSTRATING'].str.contains("ull")]
df = df[~df['REVIEWRATING'].str.contains("editorReview.rating")]
df = df[~df['REVIEWRATING'].str.contains("missing value")]
df = df[~df['URL'].str.contains("johnpacker")]
df = df[~df['URL'].str.contains("dougshop")]
print(df.describe())

print("ACTION: drop duplicates on Reviewbody")
df = df.drop_duplicates(subset='REVIEWBODY', keep="first")
print(df.describe())

print("ACTION: filter out only as english detected reviewbody")
# detect the language of the reviewbody
def detectLang(row):
    try:
        return detect(row['REVIEWBODY'])
    except:
        print("exception:", row['REVIEWBODY'])
    else:
        print("sad:", row['REVIEWBODY'])
df['LANGUAGE'] = df.apply(detectLang, axis=1)

df = df[df['LANGUAGE'] == "en"]
df.describe()

ACTION: raw data:
                                       NODE  \
count                                266232   
unique                               266230   
top     _:node36f6551f20a2dfd2ca1e4c239f898   
freq                                      2   

                                                      URL REVIEWBODY  \
count                                              266232     188381   
unique                                              37112      92156   
top     https://www.imdb.com/search/title/?title_type=...  "Null"@en   
freq                                                 1522       2415   

                  RATING REVIEWRATING BESTRATING WORSTRATING  
count             144111       241944     197734      197684  
unique             76915          233         15          19  
top     already included    "Null"@en  "Null"@en           1  
freq               18151        47380      68512       47401  
ACTION: without duplicate rows: 
                                     

,NODE,URL,REVIEWBODY,RATING,REVIEWRATING,BESTRATING,WORSTRATING,LANGUAGE
count,76961,76961,76961,76961,76961,76961,76961,76961
unique,76961,26610,76961,20641,158,13,8,1
top,_:znode8HbbjYGtCZ0BnrC7xoKpX6jE1EWuNgz,https://www.imdb.com/search/title/?title_type=...,As far as mocumentary goes this one is very co...,already included,3,10,1,en
freq,1,953,1,15511,6107,31291,37014,76961


In [94]:
df['REVIEWRATING'].value_counts().index.tolist()
# df['BESTRATING'].value_counts().index.tolist()
# df['WORSTRATING'].value_counts().index.tolist()

['"5"@en-US',
 '"5"@en',
 '"5"',
 '5',
 '"4"@en',
 '3.5',
 '"4"',
 '4',
 '"5"@en-us',
 '"4"@en-us',
 '3',
 '"8"',
 '"1"',
 '"7"',
 '" 5 "',
 '"3"',
 '"3"@en',
 '"2"',
 '4.5',
 '"4"@en-US',
 '"9"',
 '" 4 "',
 '"6"',
 '2.5',
 '"5"@vi',
 '"10"@en',
 '"8"@en-US',
 '"3"@en-US',
 '"1"@en',
 '"100%"@en',
 '0',
 '"0"@en-US',
 '"9"@en-US',
 '"5"@en-gb',
 '"5"@en-GB',
 '"95"@en-US',
 '"5"@de-DE',
 '"7"@en-US',
 '"10"',
 '"9"@en',
 '"2"@en',
 '"5"@fr',
 '"80%"@en',
 '"4.5"@en',
 '"8"@en',
 '89',
 '90',
 '"94"@en-US',
 '"1"@fr',
 '"5"@de',
 '92',
 '"4"@fr',
 '"4.5"@en-US',
 '88',
 '"6"@en-US',
 '82',
 '"4.5"',
 '"5.0"@en',
 '"100"@en',
 '2',
 '91',
 '"\\n\\t\\t\\t\\t5\\n\\t\\t\\t"@en-US',
 '"\\n\\t\\t\\t\\t5\\n\\t\\t\\t"@en-GB',
 '93',
 '86',
 '" 5"@es',
 '"5"@en-AU',
 '84',
 '"80%"',
 '87',
 '"3.8"',
 '"5"@fi',
 '"5"@da',
 '"5"@tr',
 '"7"@en',
 '"96"@en-US',
 '"5"@pt-br',
 '80',
 '85',
 '"90%"@en',
 '83',
 '"4.2"',
 '"1"@en-US',
 '"5.0"@en-US',
 '"60%"@en',
 '"0"',
 '"91"@en-GB',
 '"5"@es',
 '" 3

In [31]:
def chForm(value):
    value = re.sub(r'[a-zA-Z]|@|%| |"|-|\\', '', value)
    # return value
    if value != "":
        value = float(value)
    else:
        value = float(0)
    return round(value)
df['REVIEWRATING_new'] = df.apply(lambda row: chForm(row['REVIEWRATING']), axis=1)
df['BESTRATING_new']= df.apply(lambda row: chForm(row['BESTRATING']), axis=1)
df['WORSTRATING_new']= df.apply(lambda row: chForm(row['WORSTRATING']), axis=1)
# try:
#         int(value)
#     except:
#         # print(value)

In [101]:
df['REVIEWRATING_new'].value_counts().index.tolist()
# df['BESTRATING_new'].value_counts().index.tolist()
# df['WORSTRATING_new'].value_counts().index.tolist()

[4,
 3,
 2,
 1,
 10,
 5,
 7,
 8,
 6,
 39,
 9,
 59,
 49,
 29,
 19,
 69,
 79,
 0,
 89,
 99,
 44,
 90,
 92,
 82,
 88,
 91,
 93,
 86,
 87,
 84,
 80,
 85,
 83,
 54,
 68,
 94,
 95,
 77,
 78,
 81,
 75,
 72,
 73,
 70,
 67,
 65,
 100,
 64,
 96,
 45,
 97,
 76,
 74,
 58,
 50,
 63,
 71,
 20,
 34,
 55,
 56,
 57,
 33,
 42,
 40,
 52,
 98,
 66,
 60,
 24,
 30]

In [32]:
def adjust_rating(rating, best, worst):
    if best == 4 and worst == 0:
        return rating + 1
    elif best == 5 and worst == 0:
        if rating == 0:
            return 1
    elif rating == 0:
        return 1
    elif rating == 0.0:
        return 1
    else:
        if (best-worst) == 0:
            print(rating, best, worst)
        else:
            rating = round(5*(rating/(best-worst)))
            if rating > 5:
                rating = 5.0
            if rating == 0 or rating == 0.0:
                return 1
            else:
                return rating
df['REVIEWRATING_adj'] = df.apply(lambda row: adjust_rating(row['REVIEWRATING_new'], row['BESTRATING_new'], row['WORSTRATING_new']), axis=1)

In [26]:
df['REVIEWRATING_new'].value_counts()

5     17813
4      3230
3       635
8       371
1       286
      ...  
54        1
40        1
61        1
16        1
44        1
Name: REVIEWRATING_new, Length: 69, dtype: int64

In [28]:
(df['REVIEWRATING_adj'] == 2).sum()

134

In [33]:
print(df['REVIEWRATING_adj'].value_counts(normalize=True))
print(df['REVIEWRATING_adj'].value_counts())

4.0    0.225573
2.0    0.213543
3.0    0.207258
5.0    0.202199
1.0    0.151427
Name: REVIEWRATING_adj, dtype: float64
4.0    16763
2.0    15869
3.0    15402
5.0    15026
1.0    11253
Name: REVIEWRATING_adj, dtype: int64


In [29]:
df.describe()

,REVIEWRATING_new,BESTRATING_new,WORSTRATING_new,REVIEWRATING_adj
count,24293.000000,24293.000000,24293.00000,19550.000000
mean,7.915078,9.006833,0.77829,4.835090
std,15.596449,18.576426,0.41886,0.591529
min,0.000000,1.000000,0.00000,1.000000
25%,5.000000,5.000000,1.00000,5.000000
50%,5.000000,5.000000,1.00000,5.000000
75%,5.000000,5.000000,1.00000,5.000000
max,100.000000,100.000000,6.00000,5.000000


In [106]:
# finding out where the reviews come from (domain specific)
# def getNetloc(row):
#     try:
#         return urlparse(row['URL']).netloc
#     except:
#         print("expection: ", row['URL'])
#     else:
#         print("sad", row['URL'])
#
# df['netloc'] = df.apply(getNetloc, axis = 1)
# df.head()
# df['netloc'].value_counts().index.to_list()
# phone_lst = ['smartphone', 'phone',
#              'phone case', 'phone cable', 'phone charger', 'phone mount',
#              'cell phone', 'mobile phone']
#
#
# # df_phone = df_en[df_en['REVIEWBODY'].isin(phone_lst)]
# # print(df_phone.head())
# # LANGUAGE'] == "en"]
# df_en.head()